In [ ]:
import pandas as pd
import numpy as np

import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, FunctionTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import plotly.graph_objects as go

from IPython.display import display
from google.colab import files


from prophet import Prophet

from google.colab import auth
from google.cloud import bigquery

In [ ]:
# — Celda 2: Detección de anomalías con Prophet —

# Esta función toma el resultado de un modelo Prophet (un DataFrame con predicciones)
# y marca en qué puntos hay anomalías: 1 si está muy por encima, -1 si está muy por debajo, 0 si es normal.
# También calcula la importancia del desvío.
def detect_anomalies(forecast):
    """
    Toma el resultado de Prophet (forecast) y añade dos columnas:
      • 'anomaly':  1 si el valor real supera el límite superior,
                    -1 si está por debajo del límite inferior,
                     0 en caso contrario.
      • 'importance': magnitud del desvío relativo al valor real.
    Devuelve un DataFrame con esas nuevas columnas.
    """
    # Copiamos solo las columnas que nos interesan
    forecasted = forecast[['ds', 'trend', 'yhat', 'yhat_lower', 'yhat_upper', 'fact']].copy()

    # Inicializamos todos los puntos como "no anomalía"
    forecasted['anomaly'] = 0
    # Marcamos como anomalía positiva los casos por encima del límite superior
    forecasted.loc[forecasted['fact'] > forecasted['yhat_upper'], 'anomaly'] = 1
    # Marcamos como anomalía negativa los casos por debajo del límite inferior
    forecasted.loc[forecasted['fact'] < forecasted['yhat_lower'], 'anomaly'] = -1

    # Ahora calculamos la "importancia" del desvío para cada anomalía
    forecasted['importance'] = 0
    # Para anomalías positivas, (real - límite superior) / real
    mask_pos = forecasted['anomaly'] == 1
    forecasted.loc[mask_pos, 'importance'] = (
        (forecasted.loc[mask_pos, 'fact'] - forecasted.loc[mask_pos, 'yhat_upper'])
        / forecasted.loc[mask_pos, 'fact']
    )
    # Para anomalías negativas, (límite inferior - real) / real
    mask_neg = forecasted['anomaly'] == -1
    forecasted.loc[mask_neg, 'importance'] = (
        (forecasted.loc[mask_neg, 'yhat_lower'] - forecasted.loc[mask_neg, 'fact'])
        / forecasted.loc[mask_neg, 'fact']
    )

    return forecasted

# Esta función entrena un modelo Prophet sobre la serie de tiempo,
# predice sobre el histórico y luego detecta anomalías usando la función anterior.
# Devuelve sólo los puntos detectados como outliers.
def outliers_detection(df):
    """
    Ajusta un modelo Prophet sobre df (con columnas [fecha, valor]),
    genera el forecast y luego aplica detect_anomalies para obtener
    solo los puntos que resultaron ser outliers.
    """
    # Renombramos las columnas para Prophet: ds=fecha, y=valor
    df.columns = ['ds', 'y']

    # 1) Entrenamos Prophet
    m = Prophet()
    m = m.fit(df)

    # 2) Hacemos predicción sobre todo el histórico
    forecast = m.predict(df)
    # Guardamos el valor real para comparar
    forecast['fact'] = df['y'].reset_index(drop=True)

    # 3) Identificamos anomalías
    pred = detect_anomalies(forecast)

    # 4) Devolvemos solo las filas marcadas como anomalía
    outliers = pred[pred['anomaly'] != 0]
    return outliers

# — Conversión de cadenas a datetime —
# Esta función convierte una columna de texto (fechas) a formato datetime, usando el formato especificado.
def toDate(dataSet, atributeName, formato):
    """
    Convierte la columna dataSet[atributeName] de texto a tipo datetime
    usando el formato indicado.
    """
    dataSet[atributeName] = dataSet[atributeName].astype(str)
    return pd.to_datetime(dataSet[atributeName], format=formato)

# — Métricas de error —
# Estas funciones calculan métricas clásicas de error para modelos de regresión:
# MAPE: Error porcentual medio absoluto
# RMSE: Raíz del error cuadrático medio
# MAE: Error absoluto medio
# correlation: correlación entre la serie real y la predicción desplazada un paso (para ver tendencias)
def mape(y_true, y_pred):
    """Mean Absolute Percentage Error (porcentaje de error medio)."""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def rmse(y_true, y_pred):
    """Root Mean Squared Error (raíz del error cuadrático medio)."""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.sqrt(np.mean((y_true - y_pred)**2))

def mae(y_true, y_pred):
    """Mean Absolute Error (error absoluto medio)."""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs(y_true - y_pred))

def correlation(y_true, y_pred):
    """
    Correlación entre y_pred desplazado 1 paso y y_true original,
    para ver si predice la tendencia de un paso al siguiente.
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.corrcoef(y_true[:-1], y_pred[1:])[0, 1]

# — Validación cruzada tipo sliding window con Prophet —
# Esta función divide la serie de tiempo en varios "folds" para validación cruzada,
# entrena Prophet en cada fold y recopila las métricas de error en train y test.
def cross_validation_prophet(data, prophet_model, scal, init_split,
                             folds, params, freq, lista_exogenas,
                             variable, verbose=0):
    """
    Divide la serie en varias ventanas de entrenamiento/test,
    ajusta Prophet en cada ventana y recoge errores.
    """
    start = time.time()

    # Calculamos tamaño inicial de train y resto para folds
    init_train_size = int(init_split * len(data))
    corr = (len(data) - init_train_size) % folds
    init_train_size += corr
    fold_size = (len(data) - init_train_size) // folds

    # Generamos listas de dataframes para train/test por fold
    df_train, df_test = [], []
    for i in range(folds):
        df_train.append(data[i*fold_size : init_train_size + i*fold_size])
        df_test.append(data[init_train_size + i*fold_size :
                             init_train_size + (i+1)*fold_size])

    all_errors_train, all_errors_test = [], []
    for i in range(folds):
        if verbose:
            print(f"Fold {i+1}/{folds} (train={len(df_train[i])})")
        # Entrenamos y predecimos
        model, predict = prophet_model(
            df_train[i], df_test[i], params, freq, lista_exogenas
        )
        # Medimos errores train/test
        err_test, err_train = evaluation_dataframe(
            predict, df_train[i], df_test[i], scal, variable, verbose
        )
        all_errors_test.append(err_test)
        all_errors_train.append(err_train)

    if verbose:
        print("Tiempo total:", round(time.time() - start, 1), "segundos")
    return all_errors_test, all_errors_train

# Esta función evalúa las predicciones de Prophet, calculando las métricas (MAPE, RMSE, MAE)
# tanto para el periodo de entrenamiento como de validación, desescalando los valores si es necesario.
def evaluation_dataframe(predict, df_train, df_test, scal, variable, verbose=0):
    """
    Dado el forecast completo de Prophet (predict),
    calcula MAPE, RMSE y MAE en train y test (ya desescalando).
    """
    # Seleccionamos predicciones de test vs train según fechas
    mask_test  = predict['ds'] > df_train['ds'].iloc[-1]
    mask_train = ~mask_test

    # Valores reales y predichos (ya en escala original)
    real_test  = scal.inverse_transform(df_test[variable].values.reshape(-1,1)).flatten()
    pred_test  = scal.inverse_transform(predict.loc[mask_test, 'yhat'].values.reshape(-1,1)).flatten()
    real_train = scal.inverse_transform(df_train['y'].values.reshape(-1,1)).flatten()
    pred_train = scal.inverse_transform(predict.loc[mask_train, 'yhat'].values.reshape(-1,1)).flatten()

    # Calculamos métricas
    mape_test = mape(real_test, pred_test)
    rmse_test = rmse(real_test, pred_test)
    mae_test  = mae(real_test, pred_test)

    mape_train = mape(real_train, pred_train)
    rmse_train = rmse(real_train, pred_train)
    mae_train  = mae(real_train, pred_train)

    if verbose:
        print(f" Test → MAPE: {mape_test:.2f}%, RMSE: {rmse_test:.2f}, MAE: {mae_test:.2f}")
        print(f"Train → MAPE: {mape_train:.2f}%, RMSE: {rmse_train:.2f}, MAE: {mae_train:.2f}")

    return [mape_test, rmse_test, mae_test], [mape_train, rmse_train, mae_train]

# — Construye combinaciones de parámetros para grid search —
# Esta función toma un diccionario de listas de valores de hiperparámetros y genera todas las combinaciones posibles
# para probar en un grid search (búsqueda de hiperparámetros).
def create_param_combinations(**param_dict):
    """
    Recibe listas de valores para cada parámetro en param_dict
    y devuelve un DataFrame con todas las combinaciones posibles.
    """
    import itertools
    combos = list(itertools.product(*param_dict.values()))
    return pd.DataFrame(combos, columns=param_dict.keys())

# Esta función busca la clave de un diccionario cuyo valor es el valor pasado.
def get_key(val, my_dict):
    """Busca la clave en my_dict cuyo valor sea val."""
    for k, v in my_dict.items():
        if v == val:
            return k
    return None


In [ ]:
def cargar_datos(query):
    """Función para cargar datos desde BigQuery."""
    client = bigquery.Client()
    datos = client.query(query).to_dataframe()

    #if 'Date' in datos.columns:
    #    datos['Date'] = pd.to_datetime(datos['Date'])
    #elif 'fecha' in datos.columns:
    #    datos['fecha'] = pd.to_datetime(datos['fecha'])
    return datos

In [ ]:
# — Celda 2.1: Preparación de datos (data_preparation) —
def data_preparation(
    bool_outliers,
    inital_date,
    val_size,
    scaler,
    lista_exogenas,
    variable,
    variable_fecha,
    campana_especifica,
    verbose=0
):

    # 1) Leer CSV y filtrar campaña
    query_sabana = "SELECT * FROM `ga4-advance-analytics-alk-ktr.ZONE_STAGGING.forecast_tabla_final`"
    df = cargar_datos(query_sabana)

    df = df[df["Campaign"] == campana_especifica][[variable_fecha, variable]]
    df[variable_fecha] = pd.to_datetime(df[variable_fecha])

    # 2) Rellenar huecos y crear índice diario
    df.set_index(variable_fecha, inplace=True)
    rango_fechas = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
    df_completo = df.reindex(rango_fechas)
    df_completo[variable].interpolate(method='linear', inplace=True)

    # —— Guardamos antes de escalar — DataFrame “crudo” con valores reales diarios
    df_raw = df_completo.copy()

    # 3) Volver a formato largo para procesamiento
    df = df_completo.reset_index()
    df.columns = [variable_fecha, variable]

    # 4) Separar validación
    size_val = len(df) - int(val_size * len(df))

    # 5) Detección y corrección de outliers
    if bool_outliers:
        outliers = outliers_detection(df[[variable_fecha, variable]])
        for sign in (-1, 1):
            for date in outliers[outliers['anomaly'] == sign]['ds']:
                replacement = 'yhat_lower' if sign == -1 else 'yhat_upper'
                df.loc[df[variable_fecha] == date, variable] = \
                    outliers.loc[outliers['ds'] == date, replacement].values

    # 6) Seleccionar columnas (target + exógenas)
    df_filt = df[[variable_fecha, variable] + lista_exogenas]

    # 7) Rellenar exógenas faltantes
    for feat in lista_exogenas:
        df_filt[feat].interpolate(inplace=True)
        df_filt[feat].fillna(0, inplace=True)

    # 8) Escalar exógenas y guardar scalers
    scalers_exog = []
    for feat in lista_exogenas:
        sc = StandardScaler()
        df_filt[feat] = sc.fit_transform(df_filt[[feat]])
        scalers_exog.append(sc)

    # 9) Escalar target según parámetro
    if scaler == 'standard':
        scal = StandardScaler()
    elif scaler == 'exp':
        scal = FunctionTransformer(np.log, inverse_func=np.exp)
    elif scaler == 'minMax':
        scal = MinMaxScaler(feature_range=(-1, 1))
    else:
        scal = FunctionTransformer(None, inverse_func=None)
    df_filt[variable] = scal.fit_transform(df_filt[[variable]])

    # 10) Dividir entrenamiento y validación
    data_eva = df_filt.iloc[:size_val].copy()
    data_eva = data_eva[data_eva[variable_fecha] >= inital_date].set_index(variable_fecha)
    df_filt  = df_filt[df_filt[variable_fecha] >= inital_date].set_index(variable_fecha)

    if verbose:
        print(f"Full data: {len(df_filt)} rows")
        print(f"Train rows: {len(data_eva)}")
        print(f"Validation rows: {len(df_filt) - len(data_eva)}")

    # 11) Devolver: evaluación, scaler del target, data completa escalada, data cruda
    return data_eva, scal, df_filt, df_raw


In [ ]:
def guardar_en_bigquery(df, tabla_destino):
    """Función para guardar un DataFrame en una tabla de BigQuery."""
    df.to_gbq(destination_table=tabla_destino, if_exists='replace')
    print(f'DataFrame guardado en BigQuery en la tabla: {tabla_destino}')

# Main

In [ ]:
#Celda 3

#Main

#Variables de entrada
variable = "impressions"
variable_fecha = "Date"
campana_especifica = "AK_COL_MAX_PEF_CPC_AON_TLP_Apple_Feb20_EXP_FEB"
val_size = 0.05
bool_outliers = True  # True or False
lista_exogenas = []
scaler = "standard" #"exp" #None, exp
verbose = 1
inital_date = '2021-01-01'
granularity = 'daily'
split = 0.7

# variables frecuencia
freq_dic = {'daily':'D','hourly':'H','monthly':'MS'}
freq = freq_dic[granularity]

In [ ]:
# — Celda 4: invocar data_preparation y preparar ambos DataFrames —

data_eva, scal, df_complete_scaled, df_complete_raw = data_preparation(
    bool_outliers,
    inital_date,
    val_size,
    scaler,
    lista_exogenas,
    variable,
    variable_fecha,
    campana_especifica,
    verbose
)

# 1) Resetear índices
data_eva.reset_index(inplace=True)
df_complete_scaled.reset_index(inplace=True)

# 2) Reiniciar índice en raw y renombrar la columna de fecha
df_complete_raw = df_complete_raw.reset_index().rename(
    columns={'index': variable_fecha}
)

# 3) Comprobar
print("data_eva columns:", data_eva.columns.tolist())
print("df_complete_scaled columns:", df_complete_scaled.columns.tolist())
print("df_complete_raw columns:", df_complete_raw.columns.tolist())
display(df_complete_raw.head())


<ipython-input-23-fa9c8f80b193>:25: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpvzcovecq/52a919gp.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpvzcovecq/06o9k7vm.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=84185', 'data', 'file=/t

Full data: 888 rows
Train rows: 844
Validation rows: 44
data_eva columns: ['Date', 'impressions']
df_complete_scaled columns: ['Date', 'impressions']
df_complete_raw columns: ['Date', 'impressions']


<ipython-input-21-d646609e73e1>:29: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '<FloatingArray>
[  0.22633151741520563,  0.003977307993484989,   0.21860123855743566,
   0.30314649148209366,    0.1729314087600799,   0.43709632799689946,
   0.12400889986728274,    0.3001706836157842,    0.5571623814275025,
    0.5886098287274693,    0.2550553253204914, 0.0025794656712398857,
   0.08305144621057489,   0.11116807718097318,   0.21385928969334586,
   0.11887259248130799,   0.05548140844754237,   0.02074069498103563,
   0.09159907832865417,   0.03928734046293071, 0.0011165890836703218,
  0.007009103193546815,   0.03685158063072119,   0.05370791932399089,
    0.0227295422141736,   0.11461343680779887,   0.04002680708648698,
    0.0404980638918219,     0.159340621950308,    0.1949645114255752,
   0.21516785089063636,    0.2871782128394228,   0.23903084649378453,
    0.3379253721667928,   0.22830919946690545,  

,Date,impressions
0,2022-07-28,101562.0
1,2022-07-29,252747.0
2,2022-07-30,342950.0
3,2022-07-31,238241.0
4,2022-08-01,40786.0


In [ ]:
# — Celda 5: Forecast XGBoost con variables de calendario (30 días) —

# 1) Parámetros generales
HORIZON    = 30                         # días a predecir
freq       = 'D'                        # frecuencia diaria
var        = variable                   # p.ej. "Impressions"
datecol    = variable_fecha             # p.ej. "Date"
exogs = [
    'dayofweek','dayofmonth','month','quarter','dayofyear','is_weekend',
    'dow_sin','dow_cos','dom_sin','dom_cos','m_sin','m_cos'
]

# 2) Prepara el DataFrame de entrenamiento
#    Usamos df_complete_scaled que devolviste en la Celda 4
df = df_complete_scaled[[datecol, var]].copy()
df[datecol] = pd.to_datetime(df[datecol])
df.set_index(datecol, inplace=True)
df = df.asfreq(freq)                    # asegura índice diario
df[var] = df[var].interpolate('linear') # rellena huecos

# 3) Generación de exógenas calendario
df['dayofweek']  = df.index.dayofweek
df['dayofmonth'] = df.index.day
df['month']      = df.index.month
df['quarter']    = df.index.quarter
df['dayofyear']  = df.index.dayofyear
df['is_weekend'] = (df.index.weekday >= 5).astype(int)

# 4) Codificación cíclica
df['dow_sin'] = np.sin(2*np.pi*df.index.dayofweek    / 7)
df['dow_cos'] = np.cos(2*np.pi*df.index.dayofweek    / 7)
df['dom_sin'] = np.sin(2*np.pi*df.index.day           / df.index.days_in_month.max())
df['dom_cos'] = np.cos(2*np.pi*df.index.day           / df.index.days_in_month.max())
df['m_sin']   = np.sin(2*np.pi*df.index.month         / 12)
df['m_cos']   = np.cos(2*np.pi*df.index.month         / 12)

# 5) Guardar la serie antes de escalar (útil para graficar en escala real)
df_real = df[[var]].copy()

# 6) Escalado
sc_target = StandardScaler()
df[var] = sc_target.fit_transform(df[[var]])
scalers_exog = {}
for feat in exogs:
    sc = StandardScaler()
    df[feat] = sc.fit_transform(df[[feat]])
    scalers_exog[feat] = sc

# 7) Entrenamiento de XGBoost
X     = df[exogs]
y     = df[var]
model = xgb.XGBRegressor(objective='reg:squarederror')
model.fit(X, y)

# 8) Preparar exógenas futuras para los próximos 30 días
last_date    = df.index[-1]
future_dates = pd.date_range(last_date + pd.Timedelta(1,'D'), periods=HORIZON, freq=freq)
X_fut = pd.DataFrame(index=future_dates)
for feat in exogs:
    if feat == 'dayofweek':    raw = future_dates.dayofweek
    elif feat == 'dayofmonth': raw = future_dates.day
    elif feat == 'month':      raw = future_dates.month
    elif feat == 'quarter':    raw = future_dates.quarter
    elif feat == 'dayofyear':  raw = future_dates.dayofyear
    elif feat == 'is_weekend': raw = (future_dates.weekday >= 5).astype(int)
    elif feat == 'dow_sin':    raw = np.sin(2*np.pi*future_dates.dayofweek/7)
    elif feat == 'dow_cos':    raw = np.cos(2*np.pi*future_dates.dayofweek/7)
    elif feat == 'dom_sin':    raw = np.sin(2*np.pi*future_dates.day/future_dates.days_in_month.max())
    elif feat == 'dom_cos':    raw = np.cos(2*np.pi*future_dates.day/future_dates.days_in_month.max())
    elif feat == 'm_sin':      raw = np.sin(2*np.pi*future_dates.month/12)
    elif feat == 'm_cos':      raw = np.cos(2*np.pi*future_dates.month/12)
    tmp = pd.DataFrame({feat: raw}, index=future_dates)
    X_fut[feat] = scalers_exog[feat].transform(tmp).ravel()

# 9) Predecir y desescalar
y_scaled = model.predict(X_fut)
y_pred    = sc_target.inverse_transform(y_scaled.reshape(-1,1)).flatten()

# 10) Resultado final
df_forecast = pd.DataFrame({'Date': future_dates, 'Predicción': y_pred})


In [ ]:
# — Celda 7: Ingeniería de features para hold-out —

# Esta celda prepara las "features" (variables de entrada) para entrenar y validar el modelo en un esquema hold-out (entrenamiento/prueba).

# Partimos de data_eva, que tiene como índice la columna Date (fecha de cada registro).
feats = data_eva.copy()

# Nos aseguramos de que la columna Date sea tipo datetime, así se pueden calcular fácilmente los lags y variables de calendario.
feats['Date'] = pd.to_datetime(feats['Date'])
feats = feats.set_index('Date')

# 1) Generar lags (retrasos de la variable objetivo)
# Los lags son los valores de la variable de días anteriores.
# Aquí se generan 7 nuevas columnas, cada una con el valor de hace 1, 2, ..., 7 días.
for lag in range(1, 8):
    feats[f'lag_{lag}'] = feats[variable].shift(lag)

# Rolling means (promedios móviles)
# Calcula el promedio de los últimos 7 y 14 días para capturar tendencias cortas y medias.
feats['roll_7']  = feats[variable].rolling(7).mean()
feats['roll_14'] = feats[variable].rolling(14).mean()

# Diferencias (cambios en la variable)
# diff_1: diferencia con el día anterior. diff_7: diferencia con hace una semana.
feats['diff_1'] = feats[variable].diff(1)
feats['diff_7'] = feats[variable].diff(7)

# 2) Variables de calendario cíclico
# Se añaden las transformaciones seno y coseno para los ciclos de día de la semana, día del mes y mes del año,
# igual que en celdas anteriores, para que el modelo capte la naturaleza cíclica de la serie.
feats['dow_sin'] = np.sin(2*np.pi*feats.index.dayofweek/7)
feats['dow_cos'] = np.cos(2*np.pi*feats.index.dayofweek/7)
feats['dom_sin'] = np.sin(2*np.pi*feats.index.day/feats.index.days_in_month.max())
feats['dom_cos'] = np.cos(2*np.pi*feats.index.day/feats.index.days_in_month.max())
feats['m_sin']   = np.sin(2*np.pi*feats.index.month/12)
feats['m_cos']   = np.cos(2*np.pi*feats.index.month/12)

# 3) Eliminar filas con valores faltantes (NaN)
# Al generar lags y rolling, las primeras filas quedan sin datos (porque no hay suficientes días previos). Se eliminan.
feats = feats.dropna()

# División en train y test (hold-out)
# El dataset se divide en dos partes:
# - X_train/y_train: Para entrenar el modelo (porcentaje 'split' del total)
# - X_test/y_test: Para evaluar el modelo (el resto)
X_train = feats[: int(split * len(feats))].drop(columns=[variable])
y_train = feats[: int(split * len(feats))][variable]
X_test  = feats[int(split * len(feats)):].drop(columns=[variable])
y_test  = feats[int(split * len(feats)):][variable]


# Single XGBOOST

In [ ]:
# — Celda 8.1: Hold-out con EarlyStopping usando xgb.train —

# El objetivo de esta celda es entrenar un modelo XGBoost usando validación hold-out,
# y aprovechar EarlyStopping para evitar sobreentrenar (overfitting).

# —> Eliminar la columna Date si existe, porque solo se necesitan las features numéricas para el modelo.
if 'Date' in X_train.columns:
    X_train = X_train.drop(columns=['Date'])
    X_test  = X_test .drop(columns=['Date'])

# 1) Preparo DMatrix para entrenamiento y validación.
# DMatrix es la estructura de datos optimizada que usa XGBoost.
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_test,  label=y_test)

# 2) Defino los parámetros principales del modelo XGBoost.
params = {
    'objective':      'reg:squarederror',  # Problema de regresión (números continuos)
    'eval_metric':    'rmse',              # Usar RMSE (raíz error cuadrático medio) como métrica de evaluación
    'tree_method':    'hist',              # Método eficiente para grandes datasets
    'seed':           123                  # Semilla para reproducibilidad
}

# 3) Entreno el modelo con early stopping.
# El modelo se entrena hasta 1000 iteraciones, pero si después de 50 no mejora en validación, se detiene.
# Esto previene que el modelo aprenda "demasiado" del train y se equivoque más en test (overfitting).
bst = xgb.train(
    params,
    dtrain,
    num_boost_round=1000,                       # Número máximo de árboles (rondas)
    evals=[(dtrain, 'train'), (dvalid, 'valid')],# Validación en train y test
    early_stopping_rounds=50,                   # Si no mejora en 50 rondas, se detiene
    verbose_eval=50                             # Imprime métricas cada 50 rondas
)

# 4) Predicción en test usando el mejor número de árboles encontrados por early stopping.
y_test_pred = bst.predict(dvalid, iteration_range=(0, bst.best_iteration))

# 5) Desescalado: se convierten los resultados de vuelta a la escala original de la variable objetivo,
# usando el "scaler" que se ajustó en la preparación de datos.
y_test_real      = scal.inverse_transform(y_test .values.reshape(-1,1)).flatten()
y_test_pred_real = scal.inverse_transform(y_test_pred.reshape(-1,1)).flatten()

# 6) Cálculo de métricas para evaluar el desempeño del modelo:
# - MSE  : error cuadrático medio
# - RMSE : raíz del error cuadrático medio
# - MAE  : error absoluto medio
# - MAPE : porcentaje de error medio absoluto
# - R²   : coeficiente de determinación (qué tanto explica el modelo la variación de los datos)
mse   = mean_squared_error(y_test_real, y_test_pred_real)
rmse  = np.sqrt(mse)
mae   = mean_absolute_error(y_test_real, y_test_pred_real)
mape  = np.mean(np.abs((y_test_real - y_test_pred_real) / y_test_real)) * 100
r2    = r2_score(y_test_real, y_test_pred_real)

# Imprime las métricas para revisar el rendimiento en test.
print("🔧 Hold-out tras EarlyStopping 🔧")
print(f" MSE : {mse:,.0f}")
print(f" RMSE: {rmse:,.0f}")
print(f" MAE : {mae:,.0f}")
print(f" MAPE: {mape:.2f}%")
print(f" R²  : {r2:.3f}")


[0]	train-rmse:0.81464	valid-rmse:0.52697
[50]	train-rmse:0.00519	valid-rmse:0.17541
[100]	train-rmse:0.00091	valid-rmse:0.17514
[125]	train-rmse:0.00091	valid-rmse:0.17514
🔧 Hold-out tras EarlyStopping 🔧
 MSE : 104,401,081
 RMSE: 10,218
 MAE : 7,990
 MAPE: 6.81%
 R²  : 0.930


In [ ]:
# — Entrenar un modelo sobre las features de hold-out —
"""

model_hold = xgb.XGBRegressor(
    objective='reg:squarederror',
    tree_method='hist',
    random_state=123
)

model_hold.fit(X_train, y_train)

# Ahora sí predecimos sobre X_test que tiene lags, rolling, diff, código cíclico, etc.
y_test_pred_scaled = model_hold.predict(X_test)

# Desescala con tu scaler original
y_test_pred_real = scal.inverse_transform(y_test_pred_scaled.reshape(-1,1)).flatten()"""


"\n\nmodel_hold = xgb.XGBRegressor(\n    objective='reg:squarederror',\n    tree_method='hist',\n    random_state=123\n)\n\nmodel_hold.fit(X_train, y_train)\n\n# Ahora sí predecimos sobre X_test que tiene lags, rolling, diff, código cíclico, etc.\ny_test_pred_scaled = model_hold.predict(X_test)\n\n# Desescala con tu scaler original\ny_test_pred_real = scal.inverse_transform(y_test_pred_scaled.reshape(-1,1)).flatten()"

In [ ]:
# --- Celda 9: Training vs Test vs Test Pred en escala natural ---



# 1) Fechas de train y test desde el índice de feats
train_dates = X_train.index
test_dates  = X_test.index

# 2) # Series reales en escala natural
y_train_real = scal.inverse_transform(y_train.values.reshape(-1,1)).flatten()
y_test_real  = scal.inverse_transform(y_test.values.reshape(-1,1)).flatten()

# Métricas
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

mse  = mean_squared_error(y_test_real, y_test_pred_real)
rmse = np.sqrt(mse)
mae  = mean_absolute_error(y_test_real, y_test_pred_real)
mape = np.mean(np.abs((y_test_real - y_test_pred_real) / y_test_real))*100
r2   = r2_score(y_test_real, y_test_pred_real)

print(f"RMSE test: {rmse:.0f}, MAPE test: {mape:.2f}%, R² test: {r2:.3f}")

# Gráfico
import plotly.graph_objects as go

fig = go.Figure()

# Histórico completo (usa df de la Celda 5)
hist_real = scal.inverse_transform(df[var].values.reshape(-1,1)).flatten()
fig.add_trace(go.Scatter(
    x=df.index, y=hist_real,
    mode='lines', name='Histórico',
    line=dict(color='lightgray', width=1)
))

# Training
fig.add_trace(go.Scatter(
    x=X_train.index, y=y_train_real,
    mode='lines', name='Training',
    line=dict(color='black', width=2)
))

# Test real
fig.add_trace(go.Scatter(
    x=X_test.index, y=y_test_real,
    mode='lines', name='Test',
    line=dict(color='blue', width=2)
))

# Test predicho
fig.add_trace(go.Scatter(
    x=X_test.index, y=y_test_pred_real,
    mode='lines', name='Test Pred',
    line=dict(color='red', width=2, dash='dash')
))

fig.update_layout(
    title='XGBoost: Training vs Test vs Test Pred (escala natural)',
    xaxis_title='Fecha', yaxis_title='Impressions',
    xaxis_tickformat='%d %b\n%Y',
    width=900, height=500
)
fig.show()


RMSE test: 10218, MAPE test: 6.81%, R² test: 0.930


#Segmento de la grafica donde solo se enfoca en test sin training

In [ ]:
# — Celda 10: Zoom Test vs Predicción sin Training —

# 1) Creamos dos Series con índice datetime de X_test
test_real = pd.Series(y_test_real, index=X_test.index, name='Test real')
test_pred = pd.Series(y_test_pred_real, index=X_test.index, name='Test predicho')

# 2) Rango de fechas
start = test_real.index.min()
end   = test_real.index.max()

# 3) Graficar
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=test_real.index, y=test_real.values,
    mode='lines', name=test_real.name,
    line=dict(color='orange', width=2)
))
fig.add_trace(go.Scatter(
    x=test_pred.index, y=test_pred.values,
    mode='lines', name=test_pred.name,
    line=dict(color='green', width=2, dash='dash')
))

fig.update_layout(
    title='Zoom: Test vs Predicción',
    xaxis_title='Fecha',
    yaxis_title=variable,
    xaxis=dict(range=[start, end]),  # limitamos el eje al periodo de test
    width=800, height=400
)
fig.show()


# Salida entrenamiento

In [ ]:
# — Celda 11: Train/Test con las predicciones en escala real —

# 0) Precondiciones:
#    • scal              (el scaler original devuelto por data_preparation)
#    • sc_target         (el StandardScaler que ajustaste en Celda 5)
#    • bst, dtrain, dvalid
#    • X_train, X_test
#    • df_complete_raw   (serie cruda diaria, sin escalar)
#    • campana_especifica, granularity
#    • variable_fecha="Date", variable="Impressions"

# 1) Predicciones en la escala de sc_target:
y_train_scaled = bst.predict(dtrain)
y_test_scaled  = bst.predict(dvalid)

# 2) Primera inversión: sc_target → volvemos a la escala de data_preparation
y_train_inter = sc_target.inverse_transform(y_train_scaled.reshape(-1,1)).ravel()
y_test_inter  = sc_target.inverse_transform(y_test_scaled.reshape(-1,1)).ravel()

# 3) Segunda inversión: scal → volvemos a impresiones reales
y_train_pred = scal.inverse_transform(y_train_inter.reshape(-1,1)).ravel()
y_test_pred  = scal.inverse_transform(y_test_inter.reshape(-1,1)).ravel()

# 4) Extraer reales sin escalar de df_complete_raw
orig       = df_complete_raw.set_index(variable_fecha)[variable]
real_train = orig.reindex(X_train.index).values
real_test  = orig.reindex(X_test.index).values

# 5) Construir DataFrame de TRAIN
train_df = pd.DataFrame({
    'Date':                  X_train.index,
    'Real Impressions':      real_train,
    'Predicted Impressions': y_train_pred
})
train_df['Set'] = 'Train'

# 6) Construir DataFrame de TEST
test_df = pd.DataFrame({
    'Date':                  X_test.index,
    'Real Impressions':      real_test,
    'Predicted Impressions': y_test_pred
})
test_df['Set'] = 'Test'

# 7) Unir y enriquecer
salida_1 = pd.concat([train_df, test_df], ignore_index=True)
salida_1['Month']       = pd.to_datetime(salida_1['Date']).dt.month
salida_1['Campaign']    = campana_especifica
salida_1['Granularity'] = granularity

# 8) Reordenar y redondear
salida_1 = salida_1[[
    'Date','Month','Campaign',
    'Real Impressions','Predicted Impressions','Set'
]]
salida_1['Real Impressions']      = salida_1['Real Impressions'].round().astype(int)
salida_1['Predicted Impressions'] = salida_1['Predicted Impressions'].round().astype(int)

# 9) Mostrar la tabla final
display(salida_1)


,Date,Month,Campaign,Real Impressions,Predicted Impressions,Set
0,2022-08-10,8,AK_COL_MAX_PEF_CPC_AON_TLP_Apple_Feb20_EXP_FEB,274531,227093,Train
1,2022-08-11,8,AK_COL_MAX_PEF_CPC_AON_TLP_Apple_Feb20_EXP_FEB,125901,125917,Train
2,2022-08-12,8,AK_COL_MAX_PEF_CPC_AON_TLP_Apple_Feb20_EXP_FEB,111730,111724,Train
3,2022-08-13,8,AK_COL_MAX_PEF_CPC_AON_TLP_Apple_Feb20_EXP_FEB,87166,87224,Train
4,2022-08-14,8,AK_COL_MAX_PEF_CPC_AON_TLP_Apple_Feb20_EXP_FEB,90295,90242,Train
...,...,...,...,...,...,...
826,2024-11-13,11,AK_COL_MAX_PEF_CPC_AON_TLP_Apple_Feb20_EXP_FEB,126654,127511,Test
827,2024-11-14,11,AK_COL_MAX_PEF_CPC_AON_TLP_Apple_Feb20_EXP_FEB,154006,155658,Test
828,2024-11-15,11,AK_COL_MAX_PEF_CPC_AON_TLP_Apple_Feb20_EXP_FEB,97990,92351,Test
829,2024-11-16,11,AK_COL_MAX_PEF_CPC_AON_TLP_Apple_Feb20_EXP_FEB,178114,165882,Test


In [ ]:
guardar_en_bigquery(salida_1, 'ga4-advance-analytics-alk-ktr.ZONE_STAGGING.SF_entrenamiento_xgboost')

<ipython-input-24-430be6033eb6>:3: FutureWarning:

to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq

100%|██████████| 1/1 [00:00<00:00, 8035.07it/s]

DataFrame guardado en BigQuery en la tabla: ga4-advance-analytics-alk-ktr.ZONE_STAGGING.SF_entrenamiento_xgboost


#Forecast

In [ ]:
# — Celda 12 (corregida): Forecast hasta 31-ene-2025 y gráfico para enero —

def forecast_next_month(
    df,               # DataFrame de entrenamiento con columna 'Date' y df[var] escalada
    exogs,            # Lista de variables exógenas
    sc_target,        # StandardScaler que ajustó df[var]
    scalers_exog,     # Dict de StandardScaler para cada exógena
    model,            # XGBRegressor entrenado
    original_scaler,  # Scaler 'scal' de data_preparation para df[var]
    variable_fecha,   # Nombre de la columna de fecha ('Date')
    end_date='2025-01-31'# Fecha hasta la cual predecir
):
    # Asegurar que el índice sea datetime
    df = df.copy()
    df[variable_fecha] = pd.to_datetime(df[variable_fecha])
    df = df.set_index(variable_fecha)

    # 1) Fechas futuras
    last_date    = df.index.max()
    future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1),
                                 end=end_date, freq='D')
    X_fut = pd.DataFrame(index=future_dates)

    # 2) Generar y escalar exógenas
    for feat in exogs:
        if feat == 'dayofweek':    raw = future_dates.dayofweek
        elif feat == 'dayofmonth': raw = future_dates.day
        elif feat == 'month':      raw = future_dates.month
        elif feat == 'quarter':    raw = future_dates.quarter
        elif feat == 'dayofyear':  raw = future_dates.dayofyear
        elif feat == 'is_weekend': raw = (future_dates.weekday >= 5).astype(int)
        elif feat == 'dow_sin':    raw = np.sin(2 * np.pi * future_dates.dayofweek / 7)
        elif feat == 'dow_cos':    raw = np.cos(2 * np.pi * future_dates.dayofweek / 7)
        elif feat == 'dom_sin':    raw = np.sin(2 * np.pi * future_dates.day / future_dates.days_in_month.max())
        elif feat == 'dom_cos':    raw = np.cos(2 * np.pi * future_dates.day / future_dates.days_in_month.max())
        elif feat == 'm_sin':      raw = np.sin(2 * np.pi * future_dates.month / 12)
        elif feat == 'm_cos':      raw = np.cos(2 * np.pi * future_dates.month / 12)
        else:
            continue

        tmp = pd.DataFrame({feat: raw}, index=future_dates)
        X_fut[feat] = scalers_exog[feat].transform(tmp).ravel()

    # 3) Predicción en escala sc_target
    y_scaled = model.predict(X_fut)

    # 4) Desescalado doble
    y_inter = sc_target.inverse_transform(y_scaled.reshape(-1,1)).ravel()
    y_real  = original_scaler.inverse_transform(y_inter.reshape(-1,1)).ravel()

    return pd.DataFrame({'Date': future_dates, 'Predicción': y_real})


# — Llamada a la función y gráfico —
# Asegúrate de que df_complete_scaled tiene columna 'Date'

df_forecast = forecast_next_month(
    df_complete_scaled, exogs, sc_target, scalers_exog, model,
    original_scaler=scal,
    variable_fecha=variable_fecha,
    end_date='2025-01-31'
)

# Histórico real de enero 2025 (de df_complete_raw)
hist_jan = (
    df_complete_raw
    .set_index(variable_fecha)[variable]
    .loc['2025-01-01':'2025-01-31']
)

# Gráfico
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=hist_jan.index, y=hist_jan.values,
    mode='lines', name='Histórico Ene 2025'
))
fig.add_trace(go.Scatter(
    x=df_forecast['Date'], y=df_forecast['Predicción'],
    mode='lines', name='Predicción Ene 2025',
    line=dict(color='red', width=3)
))
fig.update_layout(
    title='Impresiones: Histórico vs Predicción Enero 2025',
    xaxis_title='Fecha',
    yaxis_title='Impresiones',
    width=900, height=500
)
fig.show()


In [ ]:
# — Celda 13: Forecast XGBoost con intervalos de confianza y gráfico —

# 0) Requisitos previos de Celda 11:
#    y_test_real, y_test_pred_real (valores reales y predichos en test, en escala real)
#    scal, sc_target, exogs, model, df_complete_scaled, variable_fecha, scalers_exog

# 1) Estimar desviación de los residuos en test
resid_std = np.std(y_test_real - y_test_pred_real)

# 2) Hacer forecast puntual
df_forecast = forecast_next_month(
    df_complete_scaled, exogs,
    sc_target, scalers_exog, model,
    original_scaler=scal,
    variable_fecha=variable_fecha,
    end_date='2025-01-31'
)

# 3) Añadir bounds ±1.96·σ
df_forecast['Lower Bound'] = df_forecast['Predicción'] - 1.96 * resid_std
df_forecast['Upper Bound'] = df_forecast['Predicción'] + 1.96 * resid_std

# 4) Gráfico con Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_forecast['Date'], y=df_forecast['Lower Bound'],
    mode='lines', name='Lower Bound',
    line=dict(color='lightblue', dash='dash')
))
fig.add_trace(go.Scatter(
    x=df_forecast['Date'], y=df_forecast['Predicción'],
    mode='lines', name='Predicción',
    line=dict(color='red', width=3)
))
fig.add_trace(go.Scatter(
    x=df_forecast['Date'], y=df_forecast['Upper Bound'],
    mode='lines', name='Upper Bound',
    line=dict(color='lightblue', dash='dash')
))

fig.update_layout(
    title='Impresiones: Histórico vs Predicción Enero 2025 con Intervalos',
    xaxis_title='Fecha', yaxis_title='Impresiones',
    width=900, height=500
)
fig.show()


In [ ]:
# — Celda 14: Mostrar tabla de predicción con bounds —

# Preparar tabla: añadir Month y redondear
tabla = df_forecast.copy()
tabla['Month'] = pd.to_datetime(tabla['Date']).dt.month
tabla = tabla[[
    'Date','Month','Predicción','Lower Bound','Upper Bound'
]].round({'Predicción':2,'Lower Bound':2,'Upper Bound':2})



In [ ]:
guardar_en_bigquery(tabla, 'ga4-advance-analytics-alk-ktr.ZONE_STAGGING.SF_prediccion_xgboost')

<ipython-input-24-430be6033eb6>:3: FutureWarning:

to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq

100%|██████████| 1/1 [00:00<00:00, 946.58it/s]

DataFrame guardado en BigQuery en la tabla: ga4-advance-analytics-alk-ktr.ZONE_STAGGING.SF_prediccion_xgboost
